<a href="https://colab.research.google.com/github/RYUNSUN/TIL_/blob/master/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0(%EB%94%A5%EB%9F%AC%EB%8B%9D)%20%ED%99%9C%EC%9A%A9%20AI%20%EC%84%A4%EA%B3%84/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%99%9C%EC%9A%A9_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/Code/Code42_02%20%EB%8F%99%EC%9A%94%EB%A7%8C%EB%93%A4%EA%B8%B0%20%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 연습문제 : 동요만들기

In [2]:
pip install konlpy

In [3]:
import numpy as np
import nltk
from konlpy.tag import *
from keras.preprocessing import *
from nltk.tokenize import *
from konlpy.corpus import kolaw
from konlpy.tag import Okt
from keras.preprocessing.text import *
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

In [0]:
def seq2dataset(seq,window_size):
  dataset=[]
  for i in range(len(seq)-window_size): # 0~49
    subset=seq[i:(i+window_size+1)]
    dataset.append([code2idx[item] for item in subset])
  
#   print(len(seq)) #length:54
#   print(dataset)
  return np.array(dataset)
    

In [6]:
dataset=seq2dataset(seq, window_size=4)
print(dataset.shape) # (50, 5)

(50, 5)


In [7]:
dataset

array([[11,  9,  2, 10,  8],
       [ 9,  2, 10,  8,  1],
       [ 2, 10,  8,  1,  7],
       [10,  8,  1,  7,  8],
       [ 8,  1,  7,  8,  9],
       [ 1,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 11],
       [ 9, 10, 11, 11,  4],
       [10, 11, 11,  4, 11],
       [11, 11,  4, 11,  9],
       [11,  4, 11,  9,  9],
       [ 4, 11,  9,  9,  9],
       [11,  9,  9,  9, 10],
       [ 9,  9,  9, 10,  8],
       [ 9,  9, 10,  8,  1],
       [ 9, 10,  8,  1,  7],
       [10,  8,  1,  7,  9],
       [ 8,  1,  7,  9, 11],
       [ 1,  7,  9, 11, 11],
       [ 7,  9, 11, 11,  9],
       [ 9, 11, 11,  9,  9],
       [11, 11,  9,  9,  2],
       [11,  9,  9,  2,  8],
       [ 9,  9,  2,  8,  8],
       [ 9,  2,  8,  8,  8],
       [ 2,  8,  8,  8,  8],
       [ 8,  8,  8,  8,  8],
       [ 8,  8,  8,  8,  9],
       [ 8,  8,  8,  9,  3],
       [ 8,  8,  9,  3,  9],
       [ 8,  9,  3,  9,  9],
       [ 9,  3,  9,  9,  9],
       [ 3,  9,  9,  9,  9],
       [ 9,  9

### 다층 퍼셉트론 모델링

In [0]:
from keras.models import *
from keras.layers import *
from keras.utils import *

In [0]:
# x,y 분리, 원핫인코딩, 정규화 작업
max_idx_value=13
xtrain=dataset[:,0:4]
ytrain=dataset[:,4]
xtrain=xtrain/float(max_idx_value) # 정규화
ytrain=np_utils.to_categorical(ytrain) # 원핫인코딩
ytrain.shape # (50, 12) 주어진 음 14개 중에서 12게 사용
one_hot_vec_size=ytrain.shape[1]

In [10]:
xtrain.shape

(50, 4)

In [11]:
ytrain.shape

(50, 12)

In [12]:
print("원 핫 벡터의 크기는 :", one_hot_vec_size)

원 핫 벡터의 크기는 : 12


In [13]:
model=Sequential()
model.add(Dense(128, input_dim=4,activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(one_hot_vec_size, activation="softmax"))

W0831 01:42:47.200598 140711157143424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0831 01:42:47.249372 140711157143424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 01:42:47.255748 140711157143424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [14]:
# 학습 알고리즘 설정
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

W0831 01:42:48.651054 140711157143424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0831 01:42:48.688294 140711157143424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3576: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
import keras
class LossHistory(keras.callbacks.Callback):
  def init(self):
    self.losses=[]
  def on_epoch_end(self, batch, logs={}): # 콜백함수 : 에폭이 끝날 때 시스템에 의해 자동 호출되는 함수 
    self.losses.append(logs.get('loss'))

In [0]:
history=LossHistory() # 클래스를 호출하면 객체가 만들어지고 losses라는 빈 리스트가 생성됨
history.init()

In [17]:
num_epochs=1000
model.fit(xtrain, ytrain, epochs=num_epochs, batch_size=1, verbose=2, callbacks=[history])
# 콜백함수: 상황에 따라 시스템이 호출하는 함수를 의미

W0831 01:42:52.913731 140711157143424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0831 01:42:52.966928 140711157143424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1000
 - 1s - loss: 2.3467 - acc: 0.3000
Epoch 2/1000
 - 0s - loss: 2.0330 - acc: 0.3400
Epoch 3/1000
 - 0s - loss: 1.9117 - acc: 0.3400
Epoch 4/1000
 - 0s - loss: 1.8912 - acc: 0.3400
Epoch 5/1000
 - 0s - loss: 1.8340 - acc: 0.3400
Epoch 6/1000
 - 0s - loss: 1.8157 - acc: 0.3400
Epoch 7/1000
 - 0s - loss: 1.8048 - acc: 0.3600
Epoch 8/1000
 - 0s - loss: 1.7654 - acc: 0.3800
Epoch 9/1000
 - 0s - loss: 1.7397 - acc: 0.4200
Epoch 10/1000
 - 0s - loss: 1.7101 - acc: 0.3800
Epoch 11/1000
 - 0s - loss: 1.7036 - acc: 0.3800
Epoch 12/1000
 - 0s - loss: 1.6807 - acc: 0.3600
Epoch 13/1000
 - 0s - loss: 1.6557 - acc: 0.4200
Epoch 14/1000
 - 0s - loss: 1.6379 - acc: 0.3800
Epoch 15/1000
 - 0s - loss: 1.6181 - acc: 0.3600
Epoch 16/1000
 - 0s - loss: 1.6022 - acc: 0.4200
Epoch 17/1000
 - 0s - loss: 1.6185 - acc: 0.4000
Epoch 18/1000
 - 0s - loss: 1.5843 - acc: 0.4000
Epoch 19/1000
 - 0s - loss: 1.5757 - acc: 0.3800
Epoch 20/1000
 - 0s - loss: 1.5703 - acc: 0.3600
Epoch 21/1000
 - 0s - loss: 1

In [19]:
scores=model.evaluate(xtrain,ytrain,batch_size=1)
scores
print("%.2f%%" % (scores[1]*100))

50/50 [==============================] - 0s 817us/step
92.00%


In [20]:
seq_out=['g8', 'e8', 'e4', 'f8']
pred_out=model.predict(xtrain, batch_size=1)
np.argmax(pred_out,1)

array([ 8,  1,  7,  9,  9, 10, 11, 11,  4, 11,  9,  9,  9, 10,  8,  1,  7,
        9, 11, 11,  9,  9,  2,  8,  8,  8,  8,  9,  9,  3,  9,  9,  9,  9,
        9,  9,  8, 11,  9,  2, 10,  8,  1,  7,  9, 11, 11,  9,  9,  2])

In [21]:
for i in range(50):    # 50: 예측개수
  idx = np.argmax(pred_out[i])
  seq_out.append(idx2code[idx])

seq_out

['g8',
 'e8',
 'e4',
 'f8',
 'd8',
 'd4',
 'c8',
 'e8',
 'e8',
 'f8',
 'g8',
 'g8',
 'g4',
 'g8',
 'e8',
 'e8',
 'e8',
 'f8',
 'd8',
 'd4',
 'c8',
 'e8',
 'g8',
 'g8',
 'e8',
 'e8',
 'e4',
 'd8',
 'd8',
 'd8',
 'd8',
 'e8',
 'e8',
 'f4',
 'e8',
 'e8',
 'e8',
 'e8',
 'e8',
 'e8',
 'd8',
 'g8',
 'e8',
 'e4',
 'f8',
 'd8',
 'd4',
 'c8',
 'e8',
 'g8',
 'g8',
 'e8',
 'e8',
 'e4']